# Tema 3 : Creación de azar (RNG)

Los humanos no somos buenos creando azar. Caemos en muchas trampas:

- "¡El número 12345 no pueder ser aleatorio!"
- "Lleva 20 años sin que la lotería acabe en 5, este año tiene que salir

Los ordenadores tampoco son mucho mejores para crear azar. Son máquinas deterministas que siempre hacen lo mismo de acuerdo con una receta, así que pueden similar azar pero no crearlo realmente

En Python, el paquete `random` se encarga de simular números al azar.

**CUIDADO: el paquete random no es una fuente de azar válida para algoritmos criptográficos**

In [ ]:
import random

In [ ]:
print(random.random())
print(random.random())
print(random.random())

In [ ]:
print(random.random())
print(random.random())
print(random.random())

En criptografía normalmente necesitamos un conjunto de bytes aleatorios, no un número. Podríamos convertir series de números reales a series de bytes así, **pero no lo hagas**.

In [ ]:
''.join(chr(int(255 * random.random())) for _ in range(0, 5)).encode()

Si ponemos una semilla, podemos repetir los números "al azar". Esto es útil, por ejemplo, en juegos, para que todos tengan las mismas cartas repartidas al azar

In [ ]:
random.seed(0.12345)
print(random.random())
print(random.random())
print(random.random())

In [ ]:
random.seed(0.12345)
print(random.random())
print(random.random())
print(random.random())

No sé cómo está programada la función `random.random()` de Python. Una manera común es utilizar la zona caótica de la [aplicación logística](https://es.wikipedia.org/wiki/Aplicaci%C3%B3n_log%C3%ADstica), como el ejemplo que viene a continuación

Fíjate que siempre necesita una semilla: si no le das una, toma el número de microsegundos de este momento según `time.time()`

In [ ]:
def rand_source(x=None):
    """ Generación de número al azar usando aplicación logística.
    
    Semilla: cualquier número real entre 0 y 1.
    
    Si no hay semilla, usa la parte decimal de time.time()
    """
    if x == None:
        import time
        _, x = divmod(time.time(), 1)
    while True:
        x = 4 * x * (1 - x)
        yield x

In [ ]:
rand = rand_source()
print(next(rand))
print(next(rand))
print(next(rand))


Si fijamos la semilla, podemos obtener siempre los mismos números aleatorios

In [ ]:
rand = rand_source(0.12345678)
print(next(rand))
print(next(rand))
print(next(rand))
rand = rand_source(0.12345678)
print(next(rand))
print(next(rand))
print(next(rand))

La ventaja de generar números aparentemente aleatorios con la aplicación logística es que es rapidísimo y muy sencillo de programar. Las desventajas son que se puede adivinar más o menos dónde caerá el próximo número dado el anterior. Ejecutad varias veces el ejemplo sin semilla: veréis que después de un número mayor de 0.9 es muy probable que paséis a un número menor de 0.3.

Además, a veces es posible conocer la semilla inicial: porque un atacante pregunta la hora justo a la vez que se crea el generador de números aleatorios, por ejemplo.

Esta forma, o alguna similar, es la manera habitual de crear números aleatorios en las librerías de programación. Pero **no es segura desde un punto de vista criptográfico**. `random.random()` o similares no se deben utilizar cuando necesitamos números aleatorios en nuestros sistemas seguros.

## Acceso a generadores seguros del sistema operativo

Casi todos los sistemas operativos permiten acceder a generadores seguros de secuencias aleatorias con algún método hardware no algorítmico.

En Linux estos generadores utilizan `/deb/urandom`, que es (más o menos!) el cifrado ChaCha20 del ruido que viene del teclado. Windows utiliza otros mecanismos similares.

Esta función, o las que ofrece el módulo Crypto, sí que se pueden utilizar en algoritmos criptográficos. Por ejemplo, se pueden utilizar para generar la clave de algoritmos de cifrado simétrico, o sus *nonce*, o sus *IV*

In [ ]:
import os
print(os.urandom(5))

In [ ]:
import Crypto.Random
Crypto.Random.get_random_bytes(5)

Fíjate que a estas funciones no les puedes pasar una semilla, porque están basadas en generadores de ruido real.

# Fuente de aletoriedad criptográfica con semilla

Una manera típica de implementar un PRNG es utilizar la fase PRNG de un cifrado de flujo. La clave es la semilla y el frujo de datos a cifrar son ceros. Por ejemplo, con ChaCha20:

In [ ]:
from Crypto.Cipher import ChaCha20
from Crypto.Random import get_random_bytes

def rand_source(x=None):
    """ Generación de número 'al azar' pero replicable usando ChaCha20.
    
    Si no hay semilla, usa una clave al azar
    """
    if x == None:
        x = get_random_bytes(32)
    cipher = ChaCha20.new(key=x, nonce=None)
    while True:
        yield cipher.encrypt(chr(0).encode())

In [ ]:
rand = rand_source()
print(next(rand))
print(next(rand))
print(next(rand))